In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_region = 3 # Set the index i for region: regions 1, 2 and 3
number_of_school = 3 # Set the index j for school of regions 1, 2 and 3 
number_of_racial = 2 # Set the index k for racial: white and black
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
distance = [[0, 3, 5], [3, 0, 4],[5, 4, 0]]  # distance miles from region i to school j  
cap_school = 300  # capability of each school 
white = [210, 210, 180] # number of students from region i in racial white
black = [120, 30, 150]  # number of students from region i in racial black
target = 100 # the number of black students should be equal to 100

In [4]:
# Set the Variable list: number of students from region i to school j in racial k(white nw and black nb)
# Set the variable nw and nb to integer number
nw = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
for i in range(number_of_region):
    for j in range(number_of_school):
        nw[i][j] = m.addVar(vtype=GRB.INTEGER, name='nw{}'.format(str(i + 1) + str(j + 1)))
    
nb = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
for i in range(number_of_region):
    for j in range(number_of_school):
        nb[i][j] = m.addVar(vtype=GRB.INTEGER, name='nb{}'.format(str(i + 1) + str(j + 1)))

In [5]:
# Caculate total numbers allocated to each region i for racial k
t1 = []
for i in range(number_of_region):
    t1.append(sum([nw[i][j] for j in range(number_of_school)]))

t2 = []
for i in range(number_of_region):
    t2.append(sum([nb[i][j] for j in range(number_of_school)]))
    
# Caculate total numbers allocated to each school j for racial k
t3 = []
for j in range(number_of_school):
    t3.append(sum([nw[i][j] for i in range(number_of_region)]))
    
t4 = []
for j in range(number_of_school):
    t4.append(sum([nb[i][j] for i in range(number_of_region)]))

In [6]:
# Set the Minimize Obijective: Total distance from region to school
m.setObjective((quicksum([nw[i][j] * distance[i][j] for i, j in [(x, y ) for x in range(number_of_region) for y in range (number_of_school)]])) + (quicksum([nb[i][j] * distance[i][j] for i, j in [(x, y ) for x in range(number_of_region) for y in range (number_of_school)]])),  GRB.MINIMIZE)

In [7]:
# Set Non Negative number of students
for i in range(number_of_region):
     for j in range(number_of_school):
            c1 = m.addConstr(nw[i][j]  >= 0)

for i in range(number_of_region):
     for j in range(number_of_school):
            c2 = m.addConstr(nb[i][j]  >= 0)

In [8]:
# Number of students for each school required as 300
c3 = []
for i in range(number_of_region):
    c3.append(m.addConstr((t1[i] + t2[i]) == cap_school))

In [9]:
# Number of black students for each school need to meet target
c4 = []
for i in range(number_of_region):
    c4.append(m.addConstr(t2[i] == target))

In [10]:
# Number of students are equal to the students from region i in racial white 
c5 = []
for j in range(number_of_school):
    c5.append(m.addConstr(t3[j] == white[j]))

In [11]:
# Number of students are equal to the students from region i in racial white 
c6 = []
for j in range(number_of_school):
    c6.append(m.addConstr(t4[j] == black[j]))

In [12]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 30 rows, 18 columns and 63 nonzeros
Model fingerprint: 0xb388f0c9
Variable types: 0 continuous, 18 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+02]
Found heuristic solution: objective 2750.0000000
Presolve removed 24 rows and 9 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros
Found heuristic solution: objective 910.0000000
Variable types: 0 continuous, 9 integer (0 binary)

Root relaxation: objective 3.500000e+02, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     350.0000000  350.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.06 seconds
Thread count was 8 (of 8 available process

In [13]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
        nw11          200 
        nw22          200 
        nw31           10 
        nw32           10 
        nw33          180 
        nb11          100 
        nb21           20 
        nb22           30 
        nb23           50 
        nb33          100 


In [14]:
# Get the Optimal Objective Value
m.ObjVal

350.0